Instalacja ClearML (jeśli nie jest zainstalowany)

In [1]:
!pip install clearml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.0 MB/s eta 0:00:00


Setup i Konfiguracja

In [2]:
# --- SETUP I IMPORTY ---
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torchvision import datasets, transforms, models
from torch.utils.data import Subset, DataLoader
from tqdm.auto import tqdm
from google.colab import drive, userdata
from clearml import Task, Logger

# Konfiguracja stałych
BASE_PATH = '/content/drive/MyDrive/Projekty_Studia'
REPO_NAME = 'dobrePraktykiProgramowania'
PROJECT_PATH = os.path.join(BASE_PATH, REPO_NAME)
RESULTS_FILE = 'raport_cnn_final.csv'

# Montowanie dysku
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Tworzenie folderów roboczych
if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)

print(f"✅ Środowisko gotowe. Ścieżka projektu: {PROJECT_PATH}")

Mounted at /content/drive
✅ Środowisko gotowe. Ścieżka projektu: /content/drive/MyDrive/Projekty_Studia/dobrePraktykiProgramowania


Logic Core

In [3]:
class ExperimentRunner:
    def __init__(self, device_type='cuda'):
        self.device = torch.device('cuda' if torch.cuda.is_available() and device_type == 'cuda' else 'cpu')
        print(f"⚙️  Inicjalizacja na: {self.device}")

    def get_data_loaders(self, batch_size=64, use_augmentation=False,
                         use_normalization=True, image_size=96, data_fraction=1.0):
        transforms_list = [transforms.Resize((image_size, image_size)), transforms.ToTensor()]
        if use_normalization:
            transforms_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))

        if use_augmentation:
            train_transforms = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15),
                transforms.ColorJitter(brightness=0.2),
            ] + transforms_list)
        else:
            train_transforms = transforms.Compose(transforms_list)

        test_transforms = transforms.Compose(transforms_list)

        trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
        testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transforms)

        if data_fraction < 1.0:
            indices = list(range(int(len(trainset) * data_fraction)))
            trainset = Subset(trainset, indices)

        return (DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0),
                DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0))

    def build_model(self, model_name='resnet50', num_classes=10, dropout_rate=0.0, use_transfer_learning=False):
        weights = None
        model = None
        if model_name == 'resnet50':
            weights = models.ResNet50_Weights.IMAGENET1K_V1 if use_transfer_learning else None
            model = models.resnet50(weights=weights)
            in_features = model.fc.in_features
            if use_transfer_learning:
                 for param in model.parameters(): param.requires_grad = False
            model.fc = nn.Sequential(nn.Dropout(p=dropout_rate), nn.Linear(in_features, num_classes))
        elif model_name == 'vgg16':
            weights = models.VGG16_Weights.IMAGENET1K_V1 if use_transfer_learning else None
            model = models.vgg16(weights=weights)
            in_features = model.classifier[6].in_features
            if use_transfer_learning:
                for param in model.features.parameters(): param.requires_grad = False
            model.classifier[6] = nn.Sequential(nn.Dropout(p=dropout_rate), nn.Linear(in_features, num_classes))
        elif model_name == 'mobilenet':
            weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V1 if use_transfer_learning else None
            model = models.mobilenet_v3_large(weights=weights)
            in_features = model.classifier[3].in_features
            if use_transfer_learning:
                for param in model.features.parameters(): param.requires_grad = False
            model.classifier[3] = nn.Sequential(nn.Dropout(p=dropout_rate), nn.Linear(in_features, num_classes))
        elif model_name == 'densenet':
            weights = models.DenseNet121_Weights.IMAGENET1K_V1 if use_transfer_learning else None
            model = models.densenet121(weights=weights)
            in_features = model.classifier.in_features
            if use_transfer_learning:
                for param in model.features.parameters(): param.requires_grad = False
            model.classifier = nn.Sequential(nn.Dropout(p=dropout_rate), nn.Linear(in_features, num_classes))

        return model.to(self.device)

    def run_training(self, model, trainloader, testloader, epochs=1, limit_batches=None, target_accuracy=None, time_limit_hours=None):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)

        start_time = time.time()
        model.train()
        history = []

        # Zmienne do estymacji czasu
        total_steps_expected = len(trainloader) * epochs
        estimation_steps = 20  # Po ilu krokach robimy estymację

        # --- CLEARML: Pobranie loggera ---
        logger = Logger.current_logger()

        print(f"   ▶ Start (Epochs: {epochs}, Cel: {target_accuracy if target_accuracy else '-'}%)")

        for epoch in range(epochs):
            running_loss = 0.0
            pbar = tqdm(trainloader, desc=f"Epoka {epoch+1}/{epochs}", leave=False)

            for i, (inputs, labels) in enumerate(pbar):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                # --- LOGIKA ESTYMACJI CZASU (np. dla CPU) ---
                current_total_step = epoch * len(trainloader) + i + 1

                if time_limit_hours and current_total_step == estimation_steps:
                    elapsed_so_far = time.time() - start_time
                    avg_time_per_batch = elapsed_so_far / estimation_steps
                    estimated_total_seconds = avg_time_per_batch * total_steps_expected
                    time_limit_seconds = time_limit_hours * 3600

                    print(f"\n⏱️ [ESTYMACJA] Średni czas na batch: {avg_time_per_batch:.4f}s")
                    print(f"⏱️ [ESTYMACJA] Przewidywany całkowity czas: {estimated_total_seconds/60:.2f} min (Limit: {time_limit_hours*60} min)")

                    if estimated_total_seconds > time_limit_seconds:
                        print(f"⚠️ Przewidywany czas > {time_limit_hours}h. Przerywam trening i zwracam estymację.")
                        return estimated_total_seconds, 0.0  # Zwracamy estymowany czas i 0% dokładności
                    else:
                        print("✅ Czas akceptowalny. Kontynuuję pełny trening...")

                # --- CLEARML: Logowanie straty (Loss) co 10 batchy ---
                if i % 10 == 0 and logger:
                    logger.report_scalar("Loss", "train", iteration=current_total_step, value=loss.item())

                # Stary mechanizm limitowania batchy (dla kompatybilności)
                if limit_batches and current_total_step >= limit_batches:
                    elapsed = time.time() - start_time
                    estimated_total = elapsed * (len(trainloader) / limit_batches)
                    return estimated_total, 0.0

            # Ewaluacja po epoce
            if testloader:
                correct, total = 0, 0
                model.eval()
                with torch.no_grad():
                    for data in testloader:
                        images, labels = data[0].to(self.device), data[1].to(self.device)
                        outputs = model(images)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                acc = 100 * correct / total
                history.append(acc)
                model.train()
                print(f"      Dokładność: {acc:.2f}%")

                # --- CLEARML: Logowanie dokładności (Accuracy) ---
                if logger:
                    logger.report_scalar("Accuracy", "test", iteration=epoch, value=acc)

                if target_accuracy and acc >= target_accuracy:
                    return time.time() - start_time, acc

        total_time = time.time() - start_time
        final_acc = history[-1] if history else 0.0
        return total_time, final_acc

Uruchomienie Eksperymentów

In [ ]:
# --- WYKONANIE TESTÓW Z CLEARML ---
from clearml import Task
import os
from google.colab import userdata

# Ustawienie zmiennych środowiskowych dla ClearML z sekretów Colab
try:
    # 1. Host API
    os.environ["CLEARML_API_HOST"] = userdata.get('CLEARML_API_HOST')

    # 2. POPRAWKA: Mapowanie Twoich nazw sekretów na nazwy wymagane przez bibliotekę ClearML
    # Biblioteka szuka "CLEARML_API_ACCESS_KEY", a w sekretach masz "CLEARML_API_KEY"
    os.environ["CLEARML_API_ACCESS_KEY"] = userdata.get('CLEARML_API_KEY')
    os.environ["CLEARML_API_SECRET_KEY"] = userdata.get('CLEARML_API_SECRET')

    print("✅ ClearML credentials loaded correctly.")
except Exception as e:
    print(f"❌ Błąd: {e}")
    print("Upewnij się, że dodałeś sekrety w ikonie 'Klucza' po lewej i włączyłeś 'Notebook access'.")
all_results = []

def log_result(category, time_s, acc, details):
    """Funkcja pomocnicza do logowania wyników do listy (dla CSV)"""
    entry = {
        'Kategoria': category,
        'Czas (s)': round(time_s, 2),
        'Dokładność (%)': round(acc, 2),
        'Szczegóły': details
    }
    all_results.append(entry)
    print(f"📊 ZAPISANO: {category} | {acc:.2f}% | {time_s:.2f}s | {details}")

def run_with_clearml(exp_name, category, params, run_function):
    """Wrapper uruchamiający eksperyment w ramach zadania ClearML"""
    # 1. Inicjalizacja zadania w ClearML
    # reuse_last_task_id=False wymusza tworzenie nowych zadań dla każdego eksperymentu
    task = Task.init(project_name="Projekt_CNN_Optymalizacja", task_name=exp_name, reuse_last_task_id=False)

    # 2. Logowanie parametrów do sekcji Configuration
    task.connect(params)

    # 3. Uruchomienie właściwej funkcji trenującej
    print(f"\n🚀 Start ClearML Task: {exp_name}")
    try:
        time_s, acc = run_function()
    except Exception as e:
        print(f"❌ Błąd podczas treningu: {e}")
        time_s, acc = 0.0, 0.0

    # 4. Zamknięcie zadania
    task.close()

    # 5. Logowanie do lokalnego CSV
    log_result(category, time_s, acc, exp_name)

runner = ExperimentRunner(device_type='cuda')

# ==========================================
# A. BASELINE (CPU vs GPU) [Wymaganie 4]
# ==========================================
print("\n--- 1. BASELINE ---")

# CPU (Inteligentna estymacja)
def run_cpu_baseline():
    runner_cpu = ExperimentRunner('cpu')
    l_cpu, _ = runner_cpu.get_data_loaders(batch_size=32)
    m_cpu = runner_cpu.build_model('resnet50')

    # Uruchamiamy z limitem 2 godzin.
    # Jeśli estymacja < 2h -> zrobi cały trening.
    # Jeśli estymacja > 2h -> przerwie po 20 batchach i zwróci przewidywany czas.
    return runner_cpu.run_training(m_cpu, l_cpu, None, epochs=1, time_limit_hours=2.0)

run_with_clearml(
    exp_name="Baseline_CPU",
    category="Baseline",
    params={'device': 'cpu', 'model': 'resnet50', 'time_limit_hours': 2.0},
    run_function=run_cpu_baseline
)

# GPU (Pełna epoka) [Wymaganie 6a]
def run_gpu_baseline():
    l_gpu, tl_gpu = runner.get_data_loaders(batch_size=64)
    m_gpu = runner.build_model('resnet50')
    return runner.run_training(m_gpu, l_gpu, tl_gpu, epochs=1)

run_with_clearml(
    exp_name="Baseline_GPU",
    category="Baseline",
    params={'device': 'gpu', 'model': 'resnet50', 'batch_size': 64},
    run_function=run_gpu_baseline
)

# ==========================================
# B. STRUKTURY SIECI [Wymaganie 18]
# ==========================================
print("\n--- 2. STRUKTURY SIECI ---")
for net in ['resnet50', 'vgg16', 'mobilenet', 'densenet']:
    def run_structure_test():
        # Mniejszy obraz (64x64) dla przyspieszenia testów wielu sieci
        l, tl = runner.get_data_loaders(batch_size=64, image_size=64)
        m = runner.build_model(net)
        return runner.run_training(m, l, tl, epochs=1)

    run_with_clearml(
        exp_name=f"Structure_{net}",
        category="Struktura",
        params={'model': net, 'image_size': 64},
        run_function=run_structure_test
    )

# ==========================================
# C. TRANSFER LEARNING [Wymaganie 7-8]
# ==========================================
print("\n--- 3. TRANSFER LEARNING ---")
def run_transfer_learning():
    # Cel: dojście do ~80%
    l_gpu, tl_gpu = runner.get_data_loaders(batch_size=64)
    m_tl = runner.build_model('resnet50', use_transfer_learning=True)
    return runner.run_training(m_tl, l_gpu, tl_gpu, epochs=20, target_accuracy=80.0)

run_with_clearml(
    exp_name="Transfer_Learning_ResNet50",
    category="Transfer Learning",
    params={'model': 'resnet50', 'pretrained': True, 'target_acc': 80.0},
    run_function=run_transfer_learning
)

# ==========================================
# D. OPTYMALIZACJE [Wymag

✅ ClearML credentials loaded correctly.
⚙️  Inicjalizacja na: cuda

--- 1. BASELINE ---
ClearML Task: created new task id=65de1f0968264ec8a8ae7196d005b838


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


ClearML results page: https://app.clear.ml/projects/a1ad3a695bd445f6ae3262f75753f8ea/experiments/65de1f0968264ec8a8ae7196d005b838/output/log

🚀 Start ClearML Task: Baseline_CPU
⚙️  Inicjalizacja na: cpu


100%|██████████| 170M/170M [00:13<00:00, 12.6MB/s]


   ▶ Start (Epochs: 1, Cel: -%)


...

      Dokładność: 79.36%


Epoka 8/20:   0%|          | 0/782 [00:00<?, ?it/s]

      Dokładność: 79.81%


Epoka 9/20:   0%|          | 0/782 [00:00<?, ?it/s]

      Dokładność: 80.12%


Kod do obsługi Git

In [ ]:
# --- 4. INTEGRACJA Z GITHUBEM (MLOPS) ---
from google.colab import userdata
import shutil

# --- KONFIGURACJA UŻYTKOWNIKA (Zmień na swoje dane!) ---
GIT_USER = "Rafalbanas"
GIT_EMAIL = "rafabanas40@gmail.com"
REPO_URL = "github.com/Rafalbanas/dobrePraktykiProgramowania.git"
COMMIT_MESSAGE = "feat: update CNN optimization results and notebook"  # Zgodne z conventional commits

def push_to_github():
    print("🚀 Rozpoczynam synchronizację z GitHub...")

    # 1. Pobranie tokenu z sekretów Colab
    try:
        git_token = userdata.get('GITHUB_TOKEN')
    except Exception:
        print("❌ BŁĄD: Nie znaleziono tokenu 'GITHUB_TOKEN' w sekretach Colab!")
        return

    # 2. Konfiguracja ścieżek
    repo_name = REPO_URL.split('/')[-1].replace('.git', '')
    repo_path = os.path.join(BASE_PATH, repo_name)

    # Skonstruowanie bezpiecznego URL z tokenem
    auth_url = f"https://{git_token}@{REPO_URL}"

    # 3. Klonowanie lub Pull
    if not os.path.exists(repo_path):
        print(f"📥 Klonowanie repozytorium do {repo_path}...")
        !git clone {auth_url} "{repo_path}"
    else:
        print("🔄 Aktualizacja repozytorium (git pull)...")
        %cd "{repo_path}"
        !git pull

    # 4. Konfiguracja tożsamości Gita
    %cd "{repo_path}"
    !git config user.email "{GIT_EMAIL}"
    !git config user.name "{GIT_USER}"

    # 5. Tworzenie .gitignore (Ważne! Ignorujemy duże pliki danych)
    gitignore_path = os.path.join(repo_path, ".gitignore")
    with open(gitignore_path, "w") as f:
        f.write("data/\n*.pth\n__pycache__/\n.ipynb_checkpoints/\n")

    # 6. Kopiowanie plików do repozytorium
    # Kopiujemy plik z wynikami CSV
    if os.path.exists(RESULTS_FILE):
        try:
            shutil.copy(RESULTS_FILE, repo_path)
            print("✅ Skopiowano raport CSV.")
        except shutil.SameFileError:
            print("ℹ️ Raport CSV już znajduje się w folderze docelowym.")

    # Kopiujemy bieżący notatnik (Zakładamy, że jest na Drive w BASE_PATH)
    # UWAGA: Zapisz notatnik (Ctrl+S) przed uruchomieniem tej komórki!
    notebook_name = "Banas_testowanie i optymalizacja sieci CNN.ipynb" # Sprawdź czy nazwa się zgadza
    source_notebook = os.path.join(BASE_PATH, notebook_name)

    if os.path.exists(source_notebook):
        try:
            shutil.copy(source_notebook, repo_path)
            print("✅ Skopiowano notatnik.")
        except shutil.SameFileError:
             print("ℹ️ Notatnik już znajduje się w folderze docelowym.")
    else:
        # Próba znalezienia notatnika w bieżącym katalogu Colab
        try:
            current_nb = [f for f in os.listdir('/content/drive/MyDrive/Colab Notebooks/') if 'CNN' in f][0] # Przybliżone szukanie
            shutil.copy(os.path.join('/content/drive/MyDrive/Colab Notebooks/', current_nb), os.path.join(repo_path, notebook_name))
            print(f"✅ Znaleziono i skopiowano: {current_nb}")
        except:
            print("⚠️ Nie znaleziono pliku .ipynb do skopiowania.")

    # 7. Git Add, Commit, Push
    !git add .
    !git commit -m "{COMMIT_MESSAGE}"
    !git push origin main
    print("🚀 Zmiany wysłane na GitHub!")

# --- TU DODAJ WYWOŁANIE FUNKCJI ---
push_to_github()